In [24]:
import pandas as pd
import seaborn as sns
import sklearn.cluster as cluster
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler as sc
import hdbscan

big_data = pd.read_csv('airbnb_coded.csv')
sc = sc()
pd.options.mode.chained_assignment = None  # default='warn'

/home/pouya/anaconda3/envs/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3,8,9,16,18,19,21,22,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
# data['property_types'] = data.property_type.map({'Apartment':0, 'House':1, 'Loft':2, 'Hotel':3, 'Condominium':4, 'Tiny house':5, 'Serviced apartment':6, 'Boutique hotel':7,
#                                            'Guest suit':8, 'Houseboat':9, 'Hostel':10, 'Villa':11, 'Townhouse':12, 'Cabin':13, 'Camper/RV':14})
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
var = 'bathrooms'
var2 = 'accommodates'
var3 = 'cleaning_fee'
var10 = 'price_x'

data = big_data[['property_type', var, var2, var3, var10]]#, 'cleaning_fee', 'security_deposit']]
# data['cleaning_fee'] = data['cleaning_fee'].fillna(data['cleaning_fee'].median())


data[var3] = data[var3].str.replace(',', '')
data[var3] = data[var3].str.replace('$', '')

big_data[var3] = big_data[var3].str.replace(',', '')
big_data[var3] = big_data[var3].str.replace('$', '')

data[var10] = data[var10].str.replace(',', '')
data[var10] = data[var10].str.replace('$', '')

big_data[var10] = big_data[var10].str.replace(',', '')
big_data[var10] = big_data[var10].str.replace('$', '')


data[var][6722] = 0
data[var][4476] = 0
data[var2][6722] = 0
data[var2][4476] = 0
data[var3][6722] = 0
data[var3][4476] = 0
# data['security_deposit'][4476] = 0
# data['security_deposit'] = data['security_deposit'].fillna(data['security_deposit'].median())
data[var] = data[var].fillna(data[var].median())
data[var2] = data[var2].fillna(data[var2].median())
data[var3] = data[var3].fillna(data[var3].median())
data[var10] = data[var10].fillna(data[var10].median())
data['property_type'] = data['property_type'].fillna('Apartment')
# data.head()
data = data.drop(data.index[14973])
big_data = big_data.drop(data.index[14973])
big_data = big_data.drop(big_data.index[6722])
big_data = big_data.drop(big_data.index[4476])
data = data.drop(data.index[6722])
data = data.drop(data.index[4476])
X = data.iloc[:, 1:-1].values
# X = ct.fit_transform(X)
# X = X.todense()
# X = np.array(X)
# X1 = data[var2].values #, data['cleaning_fee'].values, data['security_deposit'].values
# # X2 = 
# y = data['property_type'].values
# cluster_data = []
# for i in range(len(X1)):
#     cluster_data.append([y[i], X1[i]])
# cluster_data = np.array(cluster_data)
# # cluster_data = sc.fit_transform(cluster_data)
# print(len(cluster_data))

In [26]:
# label = hdbscan.HDBSCAN(min_cluster_size=450, min_samples=15).fit_predict(cluster_data)
label = cluster.KMeans(n_clusters=10).fit_predict(X)
max(label)

9

In [27]:
# label = hdbscan.HDBSCAN(min_cluster_size=450, min_samples=15).fit_predict(cluster_data)
clusterer = cluster.KMeans(n_clusters=4)
X = clusterer.fit_predict(X)
# clusterer.fit(X)
# Clusters = clusterer.predict(X)
data['listing_class'] = X
big_data['listing_class'] = X
data

,property_type,bathrooms,accommodates,cleaning_fee,price_x,listing_class
0,Apartment,1.0,2,30.00,35.00,0
1,Apartment,1.0,4,20,80.00,0
2,Apartment,1.0,4,5.00,50.00,0
3,Apartment,1.5,6,80.00,80.00,3
4,Apartment,1.0,5,50.00,180.00,1
...,...,...,...,...,...,...
18039,Apartment,1,3,20.00,90.00,0
18040,Apartment,1,5,30.00,95.00,0
18041,Apartment,1,2,10.00,39.00,0
18042,Condominium,1,2,10.00,53.00,0


In [28]:
data = data[['property_type', var, var2, var3, 'listing_class', var10]]
X = data.iloc[:, 1:-2].values
print(X)
y = data.iloc[:, -2].values
# X = ct.fit_transform(X)
# X = X.todense()
# X = np.array(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

[['1.0' '2' '30.00']
 ['1.0' '4' 20.0]
 ['1.0' '4' '5.00']
 ...
 [1.0 2 '10.00']
 [1.0 2 '10.00']
 [1.0 1 20.0]]


In [29]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth=2)
classifier = classifier.fit(X_train, y_train)
print(classifier.score(X_train, y_train))
print(classifier.score(X_test, y_test))

0.998653785239151
0.9996305191206355


In [30]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=65, weights='distance')
classifier = classifier.fit(X_train, y_train)
print(classifier.score(X_train, y_train))
print(classifier.score(X_test, y_test))

1.0
1.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=500, min_samples_leaf=1)
classifier = classifier.fit(X_train, y_train)
print(classifier.score(X_train, y_train))
print(classifier.score(X_test, y_test))

In [ ]:
#Lets now use cross validation:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(classifier, X_train, 
                          y_train, scoring='accuracy', cv=10)
print(scores)
np.mean(scores)

In [31]:
# After clustering of property types
big_data.to_csv('airbnb_coded.2.0.csv')

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR
rf = RFR(n_estimators=500, min_samples_leaf=1)
rf = rf.fit(X_train, y_train)
rf.score(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)